In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re 
from datetime import datetime as dt
%matplotlib inline

from scipy.cluster.hierarchy import linkage, fcluster, dendrogram


from sklearn import decomposition, preprocessing, metrics
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.stats import ks_2samp
from scipy.spatial.distance import cdist

In [2]:
#affichage des versions des librairies
!python --version
print('\n'.join(f'{m.__name__} - {m.__version__}'
for m in globals().values() 
if getattr(m, '__version__', None)))

Python 3.9.12
numpy - 1.21.5
pandas - 1.4.2
seaborn - 0.11.2
re - 2.2.1


In [3]:
import os
os.chdir('c:\\Users\\Matthieu\\Documents\\DAN-P9-data')  # détermination du répertoire courant

In [4]:
# Chargement des fichiers csv
Disp2017 = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
Pop2000_2018 = pd.read_csv('Population_2000_2018.csv')
# df = pd.read_csv("my_courses.csv", decimal=".", index_col=0)
pd.options.display.float_format = '{:,.2f}'.format

#### I/-Exploration des jeux de données  

In [6]:
# Affichage d'info, head et describe pour le même dataframe à la suite.
def analysis_df_start(df_input):
    display(df_input.info())
    display(df_input.head(5))
    display(df_input.describe(include="all"))

analysis_df_start(Disp2017)
analysis_df_start(Pop2000_2018)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

None

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,"4,281.00",S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,"2,302.00",S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.00,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,"6,701.00",S,Données standardisées


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
count,176600,176600,"176,600.00",176600,"176,600.00",176600,"176,600.00",176600,"176,600.00","176,600.00",176600,"176,600.00",176600,176600
unique,1,1,NaN,174,NaN,17,NaN,98,NaN,NaN,4,NaN,2,2
top,FBS,Nouveaux Bilans Alimentaire,NaN,Tchad,NaN,Disponibilité intérieure,NaN,Maïs et produits,NaN,NaN,Milliers de tonnes,NaN,S,Données standardisées
freq,176600,176600,NaN,1416,NaN,15905,NaN,2593,NaN,NaN,118487,NaN,118487,118487
mean,NaN,NaN,125.47,NaN,"3,810.05",NaN,"2,628.78",NaN,"2,017.00","2,017.00",NaN,210.55,NaN,NaN
std,NaN,NaN,72.94,NaN,"2,211.79",NaN,91.99,NaN,0.00,0.00,NaN,"4,762.05",NaN,NaN
min,NaN,NaN,1.00,NaN,645.00,NaN,"2,511.00",NaN,"2,017.00","2,017.00",NaN,"-10,388.00",NaN,NaN
25%,NaN,NaN,63.00,NaN,684.00,NaN,"2,556.00",NaN,"2,017.00","2,017.00",NaN,0.00,NaN,NaN
50%,NaN,NaN,120.00,NaN,"5,142.00",NaN,"2,612.00",NaN,"2,017.00","2,017.00",NaN,0.12,NaN,NaN
75%,NaN,NaN,188.00,NaN,"5,301.00",NaN,"2,732.00",NaN,"2,017.00","2,017.00",NaN,8.46,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

None

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,"20,779.95",X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,"21,606.99",X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,"22,600.77",X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,"23,680.87",X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,"24,726.68",X,Sources internationales sûres,NaN


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
count,4411,4411,"4,411.00",4411,"4,411.00",4411,"4,411.00",4411,"4,411.00","4,411.00",4411,"4,411.00",4411,4411,258
unique,1,1,NaN,238,NaN,1,NaN,1,NaN,NaN,1,NaN,2,2,1
top,OA,Séries temporelles annuelles,NaN,Afghanistan,NaN,Population totale,NaN,Population-Estimations,NaN,NaN,1000 personnes,NaN,X,Sources internationales sûres,"UNDESA, Population Division – World Population..."
freq,4411,4411,NaN,19,NaN,4411,NaN,4411,NaN,NaN,4411,NaN,4374,4374,258
mean,NaN,NaN,132.20,NaN,511.00,NaN,"3,010.00",NaN,"2,009.07","2,009.07",NaN,"29,630.04",NaN,NaN,NaN
std,NaN,NaN,75.85,NaN,0.00,NaN,0.00,NaN,5.48,5.48,NaN,"123,802.86",NaN,NaN,NaN
min,NaN,NaN,1.00,NaN,511.00,NaN,"3,010.00",NaN,"2,000.00","2,000.00",NaN,0.79,NaN,NaN,NaN
25%,NaN,NaN,68.00,NaN,511.00,NaN,"3,010.00",NaN,"2,004.00","2,004.00",NaN,392.19,NaN,NaN,NaN
50%,NaN,NaN,132.00,NaN,511.00,NaN,"3,010.00",NaN,"2,009.00","2,009.00",NaN,"4,764.74",NaN,NaN,NaN
75%,NaN,NaN,195.00,NaN,511.00,NaN,"3,010.00",NaN,"2,014.00","2,014.00",NaN,"18,215.48",NaN,NaN,NaN


##### I/-A/-DF "Pop2000_2018"  

In [7]:
#Suppression de colonnes inutiles
Pop2000_2018 = Pop2000_2018.drop(['Code Domaine', 'Domaine', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code année', 'Note', 'Symbole', 'Description du Symbole'], axis = 1)
Pop2000_2018.head()

,Code zone,Zone,Année,Unité,Valeur
0,2,Afghanistan,2000,1000 personnes,"20,779.95"
1,2,Afghanistan,2001,1000 personnes,"21,606.99"
2,2,Afghanistan,2002,1000 personnes,"22,600.77"
3,2,Afghanistan,2003,1000 personnes,"23,680.87"
4,2,Afghanistan,2004,1000 personnes,"24,726.68"


In [8]:
#Passage de l'unité "1000 personnes" à "personnes"
Pop2000_2018['Valeur'] = Pop2000_2018['Valeur']*1000
Pop2000_2018['Valeur'] = Pop2000_2018['Valeur'].astype(int)
Pop2000_2018 = Pop2000_2018.drop(['Unité'], axis = 1)
Pop2000_2018.head()

,Code zone,Zone,Année,Valeur
0,2,Afghanistan,2000,20779953
1,2,Afghanistan,2001,21606988
2,2,Afghanistan,2002,22600770
3,2,Afghanistan,2003,23680871
4,2,Afghanistan,2004,24726684


In [9]:
#DF filtre population en 2017
pop_2017 = Pop2000_2018[Pop2000_2018.Année == 2017].copy()
pop_2017.rename(columns={"Valeur":"habitants_2017"}, inplace=True)
pop_2017 = pop_2017.groupby(['Zone']).sum()
pop_2017.reset_index(inplace = True)
pop_2017.head()

,Zone,Code zone,Année,habitants_2017
0,Afghanistan,2,2017,36296113
1,Afrique du Sud,202,2017,57009756
2,Albanie,3,2017,2884169
3,Algérie,4,2017,41389189
4,Allemagne,79,2017,82658409


In [10]:
#DF filtre population en 2018
pop_2018 = Pop2000_2018[Pop2000_2018.Année == 2018].copy()
pop_2018.rename(columns={"Valeur":"habitants_2018"}, inplace=True)
pop_2018 = pop_2018.groupby(['Zone']).sum()
pop_2018.reset_index(inplace = True)
pop_2018.head()

,Zone,Code zone,Année,habitants_2018
0,Afghanistan,2,2018,37171921
1,Afrique du Sud,202,2018,57792518
2,Albanie,3,2018,2882740
3,Algérie,4,2018,42228408
4,Allemagne,79,2018,83124418


In [11]:
pop_2017_2018 = pd.merge(pop_2017, pop_2018, on='Zone', how='inner')
pop_2017_2018 = pop_2017_2018.drop(['Code zone_x', 'Code zone_y', 'Année_x', 'Année_y'], axis = 1)
pop_2017_2018.head()

,Zone,habitants_2017,habitants_2018
0,Afghanistan,36296113,37171921
1,Afrique du Sud,57009756,57792518
2,Albanie,2884169,2882740
3,Algérie,41389189,42228408
4,Allemagne,82658409,83124418


Le taux de variation de la population est calculé entre les années 2017 et 2018.

In [12]:
pop_2017_2018['tx_variation_population2017_18'] = round(((pop_2017_2018.habitants_2018 - pop_2017_2018.habitants_2017) / pop_2017_2018.habitants_2017)*100,2)
pop_2017_2018.head()

,Zone,habitants_2017,habitants_2018,tx_variation_population2017_18
0,Afghanistan,36296113,37171921,2.41
1,Afrique du Sud,57009756,57792518,1.37
2,Albanie,2884169,2882740,-0.05
3,Algérie,41389189,42228408,2.03
4,Allemagne,82658409,83124418,0.56


In [13]:
print("Il y a",(pop_2017_2018.isna().sum().sum()),"valeur manquante dans 'pop_2017_2018'. Il y a",(pop_2017_2018.duplicated().sum().sum()),"doublon dans 'pop_2017_2018'.")

Il y a 0 valeur manquante dans 'pop_2017_2018'. Il y a 0 doublon dans 'pop_2017_2018'.


##### I/-B/-DF "Disp2017"  

In [14]:
# Suppression de colonnes inutiles
Disp2017 = Disp2017.drop(['Code Domaine', 'Domaine', 'Code Élément', 'Code Produit', 'Code année', 'Symbole', 'Description du Symbole'], axis = 1)
Disp2017.head()

,Code zone,Zone,Élément,Produit,Année,Unité,Valeur
0,2,Afghanistan,Production,Blé et produits,2017,Milliers de tonnes,"4,281.00"
1,2,Afghanistan,Importations - Quantité,Blé et produits,2017,Milliers de tonnes,"2,302.00"
2,2,Afghanistan,Variation de stock,Blé et produits,2017,Milliers de tonnes,-119.00
3,2,Afghanistan,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes,0.00
4,2,Afghanistan,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes,"6,701.00"


In [15]:
# Filtrage pour ne garder que le produit de l'entreprise
Disp2017 = Disp2017.loc[Disp2017['Produit']=='Viande de Volailles',:]
Disp2017.head()

,Code zone,Zone,Élément,Produit,Année,Unité,Valeur
651,2,Afghanistan,Production,Viande de Volailles,2017,Milliers de tonnes,28.00
652,2,Afghanistan,Importations - Quantité,Viande de Volailles,2017,Milliers de tonnes,29.00
653,2,Afghanistan,Variation de stock,Viande de Volailles,2017,Milliers de tonnes,0.00
654,2,Afghanistan,Disponibilité intérieure,Viande de Volailles,2017,Milliers de tonnes,57.00
655,2,Afghanistan,Pertes,Viande de Volailles,2017,Milliers de tonnes,2.00


In [16]:
# Pivot de la dataframe
Disp2017 = pd.pivot_table(Disp2017, 
                          index = ['Zone'], 
                          columns = ['Élément'],
                          values = ['Valeur'], 
                          aggfunc = sum ).reset_index()

In [17]:
Disp2017.columns = ['Zone', 'Alimentation pour touristes', 'Aliments pour animaux' ,'Autres utilisations (non alimentaire)', 'Dispo_alim_volaille (kcal/pers/j)','Dispo_alim_volaille (kg/pers/an)', 
                    'Dispo MG g/pers/j', 'Dispo_prot_volaille (g/pers/j)', 'Dispo int', 'Exportation', 'Importation', 'Nourriture', 'Pertes',
                    'Production', 'Résidus','Semences', 'Traitement', 'Variation de stock']
#filtrage
Disp2017 = Disp2017[['Zone', 'Exportation' , 'Importation','Production', 'Dispo_prot_volaille (g/pers/j)']]
Disp2017.head()

,Zone,Exportation,Importation,Production,Dispo_prot_volaille (g/pers/j)
0,Afghanistan,NaN,29.00,28.00,0.54
1,Afrique du Sud,63.00,514.00,"1,667.00",14.11
2,Albanie,0.00,38.00,13.00,6.26
3,Algérie,0.00,2.00,275.00,1.97
4,Allemagne,646.00,842.00,"1,514.00",7.96


In [18]:
Disp2017['Dispo_prot_volaille (g/pers/j)'] = Disp2017['Dispo_prot_volaille (g/pers/j)']*365
Disp2017.rename(columns={"Exportation":"Exportation_2017", "Importation":"Importation_2017", "Dispo_prot_volaille (g/pers/j)":"Dispo_prot_volaille (g/pers/an)_2017", "Production":"Production_2017"},inplace=True)
Disp2017.head()

,Zone,Exportation_2017,Importation_2017,Production_2017,Dispo_prot_volaille (g/pers/an)_2017
0,Afghanistan,NaN,29.00,28.00,197.10
1,Afrique du Sud,63.00,514.00,"1,667.00","5,150.15"
2,Albanie,0.00,38.00,13.00,"2,284.90"
3,Algérie,0.00,2.00,275.00,719.05
4,Allemagne,646.00,842.00,"1,514.00","2,905.40"


##### I/-C/-Merge    

In [19]:
DisPop = Disp2017.merge(pop_2017_2018, how='left', on='Zone')

In [20]:
# Affichage d'info, head et describe pour le même dataframe à la suite.
def analysis_df_start(df_input):
    display(df_input.info())
    display(df_input.head(5))
    display(df_input.describe(include="all"))

analysis_df_start(DisPop)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Zone                                  172 non-null    object 
 1   Exportation_2017                      135 non-null    float64
 2   Importation_2017                      170 non-null    float64
 3   Production_2017                       168 non-null    float64
 4   Dispo_prot_volaille (g/pers/an)_2017  172 non-null    float64
 5   habitants_2017                        172 non-null    int32  
 6   habitants_2018                        172 non-null    int32  
 7   tx_variation_population2017_18        172 non-null    float64
dtypes: float64(5), int32(2), object(1)
memory usage: 10.8+ KB


None

,Zone,Exportation_2017,Importation_2017,Production_2017,Dispo_prot_volaille (g/pers/an)_2017,habitants_2017,habitants_2018,tx_variation_population2017_18
0,Afghanistan,NaN,29.00,28.00,197.10,36296113,37171921,2.41
1,Afrique du Sud,63.00,514.00,"1,667.00","5,150.15",57009756,57792518,1.37
2,Albanie,0.00,38.00,13.00,"2,284.90",2884169,2882740,-0.05
3,Algérie,0.00,2.00,275.00,719.05,41389189,42228408,2.03
4,Allemagne,646.00,842.00,"1,514.00","2,905.40",82658409,83124418,0.56


,Zone,Exportation_2017,Importation_2017,Production_2017,Dispo_prot_volaille (g/pers/an)_2017,habitants_2017,habitants_2018,tx_variation_population2017_18
count,172,135.00,170.00,168.00,172.00,172.00,172.00,172.00
unique,172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,132.19,89.53,725.19,"2,599.27","42,841,754.65","43,302,653.93",1.23
std,NaN,513.78,186.67,"2,501.46","2,045.26","153,063,712.48","154,236,416.18",1.11
min,NaN,0.00,0.00,0.00,14.60,"52,045.00","52,441.00",-1.75
25%,NaN,0.00,3.00,13.75,790.23,"2,874,480.25","2,862,371.00",0.41
50%,NaN,3.00,16.00,70.00,"2,374.32","9,757,833.00","9,828,518.00",1.15
75%,NaN,32.00,81.25,409.75,"3,783.22","30,138,736.00","30,989,348.50",2.01


In [21]:
DisPop.isna().sum()

Zone                                     0
Exportation_2017                        37
Importation_2017                         2
Production_2017                          4
Dispo_prot_volaille (g/pers/an)_2017     0
habitants_2017                           0
habitants_2018                           0
tx_variation_population2017_18           0
dtype: int64

#### II/-Bases complémentaires    

##### II/-A/-Stabilité  

In [22]:
stabilite = pd.read_csv('stabilite.csv')
stabilite.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2000,2000,indice,-2.44,X,Ciffre de sources internationales,NaN
1,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2002,2002,indice,-2.04,X,Ciffre de sources internationales,NaN
2,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2003,2003,indice,-2.20,X,Ciffre de sources internationales,NaN
3,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2004,2004,indice,-2.30,X,Ciffre de sources internationales,NaN
4,FS,Données de la sécurité alimentaire,2,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2005,2005,indice,-2.07,X,Ciffre de sources internationales,NaN


In [23]:
stabilite = stabilite[['Zone','Année','Produit','Valeur']]

In [24]:
sta_2000 = stabilite[stabilite.Année == 2000]
sta_2000 = sta_2000.groupby(['Zone']).sum()
sta_2000.reset_index(inplace = True)
sta_2000.head()

,Zone,Année,Valeur
0,Afghanistan,2000,-2.44
1,Afrique du Sud,2000,-0.23
2,Albanie,2000,-0.54
3,Algérie,2000,-1.43
4,Allemagne,2000,1.41


In [25]:
sta_2020 = stabilite[stabilite.Année == 2020]
sta_2020 = sta_2020.groupby(['Zone']).sum()
sta_2020.reset_index(inplace = True)
sta_2020.head()

,Zone,Année,Valeur
0,Afghanistan,2020,-2.73
1,Afrique du Sud,2020,-0.24
2,Albanie,2020,0.08
3,Algérie,2020,-0.86
4,Allemagne,2020,0.67


In [26]:
sta_2000_2020 = pd.merge(sta_2020, sta_2000, on='Zone', how='inner')
sta_2000_2020.head()

,Zone,Année_x,Valeur_x,Année_y,Valeur_y
0,Afghanistan,2020,-2.73,2000,-2.44
1,Afrique du Sud,2020,-0.24,2000,-0.23
2,Albanie,2020,0.08,2000,-0.54
3,Algérie,2020,-0.86,2000,-1.43
4,Allemagne,2020,0.67,2000,1.41


In [27]:
sta_2000_2020['tx_variation_stabilite2000_20'] = round(((sta_2000_2020.Valeur_x - (sta_2000_2020.Valeur_y)) / (sta_2000_2020.Valeur_y))*100,2)

In [28]:
sta_2000_2020.rename(columns={"Valeur_x":"stab_2020", "Valeur_y":"stab_2000"}, inplace=True)
sta_2000_2020 = sta_2000_2020.drop(["Année_x", "Année_y"], axis = 1)
sta_2000_2020.head()

,Zone,stab_2020,stab_2000,tx_variation_stabilite2000_20
0,Afghanistan,-2.73,-2.44,11.89
1,Afrique du Sud,-0.24,-0.23,4.35
2,Albanie,0.08,-0.54,-114.81
3,Algérie,-0.86,-1.43,-39.86
4,Allemagne,0.67,1.41,-52.48


##### II/-B/-Commerce de poulets entre la France et le reste du monde 2018_2021 (volume et valeur)  

In [29]:
CommercePouletsFrance2018_21 = pd.read_csv('viandePoulet18_21.csv')
CommercePouletsFrance2018_21.head()

,Code Domaine,Domaine,Code pays déclarant (M49),Pays déclarants,Code pays partenaire (M49),Pays partenaires,Code Élément,Élément,Code Produit (CPC),Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,TM,Matrices du commerce détaillées,250,France,344,Chine - RAS de Hong-Kong,5910,Exportations - Quantité,21121,"Viande, poulet, fraîche ou réfrigérée",2018,2018,tonnes,"9,922.59",A,Chiffre officiel
1,TM,Matrices du commerce détaillées,250,France,124,Canada,5910,Exportations - Quantité,21121,"Viande, poulet, fraîche ou réfrigérée",2018,2018,tonnes,99.15,A,Chiffre officiel
2,TM,Matrices du commerce détaillées,250,France,8,Albanie,5910,Exportations - Quantité,21121,"Viande, poulet, fraîche ou réfrigérée",2018,2018,tonnes,98.88,A,Chiffre officiel
3,TM,Matrices du commerce détaillées,250,France,344,Chine - RAS de Hong-Kong,5910,Exportations - Quantité,21121,"Viande, poulet, fraîche ou réfrigérée",2020,2020,tonnes,"9,656.41",A,Chiffre officiel
4,TM,Matrices du commerce détaillées,250,France,562,Niger,5910,Exportations - Quantité,21121,"Viande, poulet, fraîche ou réfrigérée",2019,2019,tonnes,960.09,A,Chiffre officiel


In [30]:
CommercePouletsFrance2018_21 = CommercePouletsFrance2018_21[['Pays partenaires', 'Élément' , 'Année','Unité', 'Valeur']]
CommercePouletsFrance2018_21.head()

,Pays partenaires,Élément,Année,Unité,Valeur
0,Chine - RAS de Hong-Kong,Exportations - Quantité,2018,tonnes,"9,922.59"
1,Canada,Exportations - Quantité,2018,tonnes,99.15
2,Albanie,Exportations - Quantité,2018,tonnes,98.88
3,Chine - RAS de Hong-Kong,Exportations - Quantité,2020,tonnes,"9,656.41"
4,Niger,Exportations - Quantité,2019,tonnes,960.09


In [31]:
Commerce2018 = CommercePouletsFrance2018_21[CommercePouletsFrance2018_21.Année == 2018].copy()
Commerce2018.reset_index(inplace = True)
Commerce2018.rename(columns={"Valeur":"Valeur_2018"}, inplace=True)
Commerce2018.head()

,index,Pays partenaires,Élément,Année,Unité,Valeur_2018
0,0,Chine - RAS de Hong-Kong,Exportations - Quantité,2018,tonnes,"9,922.59"
1,1,Canada,Exportations - Quantité,2018,tonnes,99.15
2,2,Albanie,Exportations - Quantité,2018,tonnes,98.88
3,9,Côte d'Ivoire,Exportations - Quantité,2018,tonnes,903.90
4,11,Liban,Exportations - Quantité,2018,tonnes,90.60


In [32]:
Commerce2019 = CommercePouletsFrance2018_21[CommercePouletsFrance2018_21.Année == 2019].copy()
Commerce2019.reset_index(inplace = True)
Commerce2019.rename(columns={"Valeur":"Valeur_2019"}, inplace=True)
Commerce2019.head()

,index,Pays partenaires,Élément,Année,Unité,Valeur_2019
0,4,Niger,Exportations - Quantité,2019,tonnes,960.09
1,6,Croatie,Exportations - Quantité,2019,tonnes,94.97
2,10,Oman,Exportations - Quantité,2019,tonnes,900.29
3,17,Norvège,Exportations - Quantité,2019,tonnes,9.05
4,22,Qatar,Exportations - Quantité,2019,tonnes,843.85


In [33]:
Commerce2020 = CommercePouletsFrance2018_21[CommercePouletsFrance2018_21.Année == 2020].copy()
Commerce2020.reset_index(inplace = True)
Commerce2020.rename(columns={"Valeur":"Valeur_2020"}, inplace=True)
Commerce2020.head()

,index,Pays partenaires,Élément,Année,Unité,Valeur_2020
0,3,Chine - RAS de Hong-Kong,Exportations - Quantité,2020,tonnes,"9,656.41"
1,5,Serbie,Exportations - Quantité,2020,tonnes,953.41
2,14,Norvège,Exportations - Quantité,2020,tonnes,9.47
3,15,Mali,Exportations - Quantité,2020,tonnes,9.32
4,19,Irlande,Exportations - Quantité,2020,tonnes,89.88


In [34]:
Commerce2021 = CommercePouletsFrance2018_21[CommercePouletsFrance2018_21.Année == 2021].copy()
Commerce2021.reset_index(inplace = True)
Commerce2021.rename(columns={"Valeur":"Valeur_2021"}, inplace=True)
Commerce2021.head()

,index,Pays partenaires,Élément,Année,Unité,Valeur_2021
0,7,"Chine, continentale",Exportations - Quantité,2021,tonnes,93.26
1,8,Royaume-Uni de Grande-Bretagne et d'Irlande du...,Exportations - Quantité,2021,tonnes,"9,298.51"
2,13,Saint-Kitts-et-Nevis,Exportations - Quantité,2021,tonnes,9.86
3,16,Éthiopie,Exportations - Quantité,2021,tonnes,9.13
4,18,Philippines,Exportations - Quantité,2021,tonnes,893.79


In [35]:
Commerce2018_2019 = pd.merge(Commerce2018, Commerce2019, how='outer', left_on=['Pays partenaires', 'Élément', 'Unité'], right_on = ['Pays partenaires', 'Élément', 'Unité'])
Commerce2020_2021 = pd.merge(Commerce2020, Commerce2021, how='outer', left_on=['Pays partenaires', 'Élément', 'Unité'], right_on = ['Pays partenaires', 'Élément', 'Unité'])
Commerce2018_2021 = pd.merge(Commerce2018_2019, Commerce2020_2021, how='outer', left_on=['Pays partenaires', 'Élément', 'Unité'], right_on = ['Pays partenaires', 'Élément', 'Unité'])

In [36]:
Commerce2018_2021 = Commerce2018_2021[['Pays partenaires', 'Élément' , 'Unité', 'Valeur_2018', 'Valeur_2019', 'Valeur_2020', 'Valeur_2021']]
Commerce2018_2021['tx_variation_Commerce2018_20'] = round(((Commerce2018_2021.Valeur_2020 - (Commerce2018_2021.Valeur_2018)) / (Commerce2018_2021.Valeur_2018))*100,2)
Commerce2018_2021.head()

,Pays partenaires,Élément,Unité,Valeur_2018,Valeur_2019,Valeur_2020,Valeur_2021,tx_variation_Commerce2018_20
0,Chine - RAS de Hong-Kong,Exportations - Quantité,tonnes,"9,922.59","11,600.21","9,656.41","3,128.65",-2.68
1,Canada,Exportations - Quantité,tonnes,99.15,6.47,38.03,8.10,-61.64
2,Albanie,Exportations - Quantité,tonnes,98.88,157.60,277.25,486.38,180.39
3,Côte d'Ivoire,Exportations - Quantité,tonnes,903.90,558.62,232.73,501.68,-74.25
4,Liban,Exportations - Quantité,tonnes,90.60,261.04,267.18,135.26,194.90


In [37]:
exportTonnes = Commerce2018_2021[(Commerce2018_2021.Élément == "Exportations - Quantité")&(Commerce2018_2021.Unité == "tonnes")].copy()
exportTonnes.rename(columns={"Valeur_2018":"exportTonnes_2018", "Valeur_2019":"exportTonnes_2019", "Valeur_2020":"exportTonnes_2020", "Valeur_2021":"exportTonnes_2021", "tx_variation_Commerce2018_20":"exportTonnes_tv1820"},inplace=True)
exportTonnes = exportTonnes.drop(["Élément", "Unité"], axis = 1)
exportTonnes.head()

,Pays partenaires,exportTonnes_2018,exportTonnes_2019,exportTonnes_2020,exportTonnes_2021,exportTonnes_tv1820
0,Chine - RAS de Hong-Kong,"9,922.59","11,600.21","9,656.41","3,128.65",-2.68
1,Canada,99.15,6.47,38.03,8.10,-61.64
2,Albanie,98.88,157.60,277.25,486.38,180.39
3,Côte d'Ivoire,903.90,558.62,232.73,501.68,-74.25
4,Liban,90.60,261.04,267.18,135.26,194.90


In [38]:
#Unité = 1000 US$
exportValeur = Commerce2018_2021[(Commerce2018_2021.Élément == "Exportations - Valeur")&(Commerce2018_2021.Unité == "1000 US$")].copy()
exportValeur.rename(columns={"Valeur_2018":"exportValeur_2018", "Valeur_2019":"exportValeur_2019", "Valeur_2020":"exportValeur_2020", "Valeur_2021":"exportValeur_2021", "tx_variation_Commerce2018_20":"exportValeur_tv1820"},inplace=True)
exportValeur = exportValeur.drop(["Élément", "Unité"], axis = 1)
exportValeur.head()

,Pays partenaires,exportValeur_2018,exportValeur_2019,exportValeur_2020,exportValeur_2021,exportValeur_tv1820
99,Arabie saoudite,"133,958.00","98,082.00","99,717.00","101,920.00",-25.56
100,Allemagne,"80,038.00","69,091.00","62,274.00","82,411.00",-22.19
101,Belgique,"69,250.00","64,588.00","57,872.00","66,548.00",-16.43
102,Royaume-Uni de Grande-Bretagne et d'Irlande du...,"37,484.00","43,228.00","26,945.00","36,563.00",-28.12
103,Pays-Bas,"29,359.00","28,476.00","24,967.00","40,169.00",-14.96


#### III/-Jointures et nettoyage des données    

In [39]:
#merge données commerce
comFin = pd.merge(exportValeur,exportTonnes,how='outer', on='Pays partenaires')

In [40]:
df_fin = sta_2000_2020.merge(DisPop, how='outer', on='Zone')
df_fin.rename(columns={"Zone":"Pays partenaires"},inplace=True)

In [41]:
df_final = pd.merge(df_fin,comFin,how='outer', on='Pays partenaires')

In [42]:
#19 petits pays avec des valeurs manquantes pour la colonne  "habitants_2018"
df_hab18NA = df_final[df_final["habitants_2018"].isna()]
df_hab18NA['Pays partenaires'].values

array(['Andorre', 'Bahreïn', 'Bermudes', 'Bhoutan', 'Brunéi Darussalam',
       'Burundi', 'Comores', 'Guinée équatoriale', 'Libye', 'Palestine',
       'Papouasie-Nouvelle-Guinée', 'Porto Rico', 'Qatar',
       'République arabe syrienne', 'République démocratique du Congo',
       'Seychelles', 'Singapour', 'Somalie', 'Érythrée'], dtype=object)

In [43]:
#Suppression de ces 19 pays avec des valeurs manquantes pour la colonne  "habitants_2018"
df_final1 = df_final.drop(df_hab18NA.index).reset_index(drop=True)

In [52]:
#Affichage d'infos, head et describe pour le même dataframe à la suite.
def analysis_df_start(df_input):
    display(df_input.isna().sum())
analysis_df_start(df_final)

Pays partenaires                         0
stab_2020                                6
stab_2000                                6
tx_variation_stabilite2000_20            6
Exportation_2017                        56
Importation_2017                        21
Production_2017                         23
Dispo_prot_volaille (g/pers/an)_2017    19
habitants_2017                          19
habitants_2018                          19
tx_variation_population2017_18          19
exportValeur_2018                       86
exportValeur_2019                       88
exportValeur_2020                       86
exportValeur_2021                       79
exportValeur_tv1820                     98
exportTonnes_2018                       86
exportTonnes_2019                       88
exportTonnes_2020                       86
exportTonnes_2021                       79
exportTonnes_tv1820                     98
dtype: int64

In [45]:
#Suppression des pays avec de faibles valeurs à l'export de poulets avec la France et des valeurs manquantes pour la colonne  "exportValeur_2018"
df_expVal18 = df_final1[df_final1["exportValeur_2018"].isna()]
df_expVal18['Pays partenaires'].values

array(['Afghanistan', 'Algérie', 'Antigua-et-Barbuda', 'Argentine',
       'Australie', 'Azerbaïdjan', 'Bangladesh', 'Barbade', 'Belize',
       'Bolivie (État plurinational de)', 'Botswana', 'Brésil', 'Chili',
       'Chine, Taiwan Province de', 'Colombie', 'Costa Rica', 'Cuba',
       'Dominique', 'El Salvador', 'Eswatini', 'Fidji', 'France',
       'Gambie', 'Grenade', 'Guatemala', 'Guinée-Bissau', 'Guyana',
       'Honduras', 'Inde', 'Indonésie', "Iran (République islamique d')",
       'Iraq', 'Islande', 'Israël', 'Jamaïque', 'Kenya', 'Kirghizistan',
       'Lesotho', 'Madagascar', 'Malawi', 'Mauritanie', 'Mexique',
       'Mongolie', 'Myanmar', 'Nicaragua', 'Nouvelle-Zélande', 'Népal',
       'Ouganda', 'Ouzbékistan', 'Pakistan', 'Panama', 'Paraguay',
       'Pérou', 'Rwanda', 'République de Corée',
       'République démocratique populaire lao',
       'République populaire démocratique de Corée',
       'République-Unie de Tanzanie', 'Saint-Vincent-et-les Grenadines',
       'S

In [46]:
#Suppression des lignes avec des valeurs manquantes pour la colonne  "exportValeur_2018"
df_final2 = df_final1.drop(df_expVal18.index).reset_index(drop=True)

In [47]:
display(df_final2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 21 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Pays partenaires                      96 non-null     object 
 1   stab_2020                             93 non-null     float64
 2   stab_2000                             93 non-null     float64
 3   tx_variation_stabilite2000_20         93 non-null     float64
 4   Exportation_2017                      80 non-null     float64
 5   Importation_2017                      96 non-null     float64
 6   Production_2017                       94 non-null     float64
 7   Dispo_prot_volaille (g/pers/an)_2017  96 non-null     float64
 8   habitants_2017                        96 non-null     float64
 9   habitants_2018                        96 non-null     float64
 10  tx_variation_population2017_18        96 non-null     float64
 11  exportValeur_2018    

None

In [53]:
df_final31 = df_final2.drop(['exportTonnes_tv1820', 'Exportation_2017', 'stab_2020', 'stab_2000', 'habitants_2017', 'exportTonnes_2018', 'exportTonnes_2019', 'exportTonnes_2020'], axis = 1)

In [49]:
df_final31.rename(columns={"Pays partenaires":"Pays"},inplace=True)

In [50]:
# Suppression des outliers Jordanie et République dominicaine qui comptent des valeurs 'inf'.
indexNames = df_final31[df_final31['Pays'] == 'Jordanie'].index
df_final31 = df_final31.drop(indexNames)
indexNames = df_final31[df_final31['Pays'] == 'République dominicaine'].index
df_final31 = df_final31.drop(indexNames)

print("Il y a",(len(df_final31)),"pays dans le jeu de données final.")

Il y a 94 pays dans le jeu de données final.


In [51]:
df_final31.to_csv('df_final31.csv')